# Time Phrase Curation

The starting point for this analysis are phrases in the ETCBC's [BHSA dataset](https://github.com/etcbc/bhsa) with a `function` feature value of `Time`. But these phrases are technically very similar to phrases marked with a function of `Adju` ("adjunct"), except with a further specification. The `Time` phrases in the BHSA are not likely to be "perfect" and there may be some inconsistencies. The purpose of this notebook is to curate time phrases that are used for all analyses in this project. This involves rigorously querying for anomalies and manually checking phrases that will be included in the analysis.

The custom features `head` and `nhead` are also crucial for processing various data on these time phrases (see [heads repository](https://github.com/etcbc/heads)). The `head` feature specifies a semantic phrase head while the `nhead` feature specifies nominal heads, including those subsumed under a preposition. The difference between the two features is that, for prepositional phrases, `head` links to the preposition head, whereas `nhead` links beyond the preposition (and through any chained prepositions) to the non-quantified nominal element that is governed by it. The benefit of the `nhead` is one can determine the primary semantic element within the phrase without referencing the functional prepositions.

The `head` and `nhead` features are experimental, and thus all of the identified heads need to be validated in order to be utilized further in the analysis. This notebook will also check the features for all of the time phrase tokens (tokenized surface forms). 

In [1]:
import collections, random, csv
import pandas as pd
from tf.app import use

# load BHSA
A = use('bhsa', hoist=globals(), mod='etcbc/heads/tf', check=True)
A.api.TF.load('''g_cons_utf8 
                 prs_ps prs prs_nu
                 ''', add=True)

TF app is up-to-date.
Using annotation/app-bhsa commit d3cf8f0c2ab5d690a0fda14ea31c33da5c5c8483 (=latest)
  in /Users/cody/text-fabric-data/__apps__/bhsa.
No new data release available online.
Using etcbc/bhsa/tf - c rv1.6 (=latest) in /Users/cody/text-fabric-data.
No new data release available online.
Using etcbc/phono/tf - c r1.2 (=latest) in /Users/cody/text-fabric-data.
No new data release available online.
Using etcbc/parallels/tf - c r1.2 (=latest) in /Users/cody/text-fabric-data.
No new data release available online.
Using etcbc/heads/tf - c rv.1.3 (=latest) in /Users/cody/text-fabric-data.


  0.00s loading features ...
   |     0.11s B prs                  from /Users/cody/text-fabric-data/etcbc/bhsa/tf/c
  0.12s All additional features loaded - for details use loadLog()


In [2]:
def tokenPhrase(phrasenode):
    '''Tokenizes a phrase with
    dot-separated words.
    input: phrase node number
    output: token string'''
    words = [(F.g_cons_utf8.v(w) if F.lex.v(w) != 'H' else 'ה') for w in L.d(phrasenode, 'word')]
    return '.'.join(words)

def tokenHeads(headslist):
    '''same as tokenPhrase but with list of head word nodes'''
    return '.'.join((F.g_cons_utf8.v(w) if F.lex.v(w) != 'H' else 'ה') for w in headslist)

## Phrase Tokens and Phrase Heads

This search counts all of the discrete time phrase tokens in Hebrew and gathers data about their heads. This data is exported to a spreadsheet for manual inspection. Per every token, a key of its heads is saved into a dictionary, linked to a list of phrase nodes. Tokens that have more than 1 head are suspicious, since the surface form is the same. All other tokens will be exported with their standard heads for inspection. 

In [3]:
tp_heads = collections.defaultdict(lambda: collections.defaultdict(list))
tp_nheads = collections.defaultdict(lambda: collections.defaultdict(list))
tp_count = collections.Counter()

tps = A.search('''

phrase function=Time
/with/
    word language=Hebrew
/-/

''', shallow=True)

for tp in tps:
    token = tokenPhrase(tp)
    heads_token = tokenHeads(E.head.t(tp))
    nheads_token = tokenHeads(E.nhead.t(tp))
    
    
    tp_heads[token][heads_token].append(tp)
        
    # only populate nheads with PP phrases, since nhead feature for NP is exactly the same
    if F.typ.v(tp) == 'PP':
        tp_nheads[token][nheads_token].append(tp)
        
    tp_count[token] += 1
    
suspect_heads = [tp for tp in tp_heads if len(tp_heads[tp]) > 1]
suspect_nheads = [tp for tp in tp_nheads if len(tp_nheads[tp]) > 1]

print(f'total phrase tokens 2 head mappings: {len(tp_heads)}')
print(f'total phrase tokens 2 nhead mappings: {len(tp_nheads)}')
print(f'total suspect heads: {len(suspect_heads)}')
print(f'total suspect nheads {len(suspect_nheads)}')

  0.86s 3961 results
total phrase tokens 2 head mappings: 1171
total phrase tokens 2 nhead mappings: 894
total suspect heads: 0
total suspect nheads 0


**NB**<br>
The initial run of this search found problems in the phrase: ב.ה.בקר.ב.ה.בקר. Some cases marked the second part of the phrase a parallel element, whereas others marked them as either a phrase atom specification relation (`Spec`) or a subphrase adjunct relation (`adj`). This is an inconsistent tagging on the BHSA's part. These issues were addressed in the [heads notebook](https://nbviewer.jupyter.org/github/ETCBC/heads/blob/master/phrase_heads.ipynb) of the ETCBC heads repository. The phrase in question is now correctly annotated.

### Compile Manual Inspection Spreadsheet

In [4]:
tp_heads_data = []
tp_nheads_data = []
data_header = ['token', '(n)heads_token', 'freq', 'mark', 'note', 'ex_ref', 'ex', 'ex_node', 'ex_verse']

for htp, nhtp in zip(tp_heads.keys(), tp_nheads.keys()):
    head = next(tp for tp in tp_heads[htp])
    nhead = next(tp for tp in tp_nheads[nhtp])
    head_ex = random.choice(tp_heads[htp][head])
    nhead_ex = random.choice(tp_nheads[nhtp][nhead])
    
    head_ref, nhead_ref = ['{} {}:{}'.format(*T.sectionFromNode(ex)) for ex in (head_ex, nhead_ex)]
    head_txt, nhead_txt = [T.text(ex) for ex in (head_ex, nhead_ex)]
    head_verse, nhead_verse = [T.text(L.u(ex, 'verse')[0]) for ex in (head_ex, nhead_ex)]
    
    heads_data = [htp, head, tp_count[htp], '', '', head_ref, head_txt, head_ex, head_verse]
    nheads_data = [nhtp, nhead, tp_count[nhtp], '', '', nhead_ref, nhead_txt, nhead_ex, nhead_verse]
    tp_heads_data.append(heads_data)
    tp_nheads_data.append(nheads_data)
    
tp_heads_data, tp_nheads_data = sorted(tp_heads_data), sorted(tp_nheads_data)

In [5]:
with open('manual_curation/tp_heads.csv', 'w') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(data_header)
    writer.writerows(tp_heads_data)
    
with open('manual_curation/tp_nheads.csv', 'w') as outfile:
    writer = csv.writer(outfile)
    writer.writerow(data_header)
    writer.writerows(tp_nheads_data)

# Produce Report on Manual Annotations

The manual annotations are intended to serve 2 roles: 1. to evaluate the accuracy of the head assignments on time phrases in the BHSA data, 2. to evaluate the time phrase structure in the BHSA dataset, and 3. to gain hands-on exposure to the kinds of time phrases in the dataset. This process consisted of comparing the selected heads against the surface text of the time phrase, and of reading the time phrases in the context of a verse when questions or anomalies arose. The annotation process consisted of marking a given time phrase as "g" for "good," "b" for "bad," and "?" for questionable cases. These classifications refer to both head assignments and internal structuring of the time phrases in the BHSA. The markings are often accompanied with notes: for bad or questionable entries the note explains what is wrong, for good entries the note might describe an interesting phenomenon, in some cases it might give a "light caution" about a given phrase. 

The annotations suggest that custom database is necessary to consistently represent time phrases: there are many cases in BHSA time phrases where the phrase is cleft into 2 adjacent parts whereas in the majority of the data they are kept together. This is an inconsistency that should be solved. In other cases, the notion of "phrase" is not broad enough to encompass the full range of expressions that can mark time. For instance, several time phrases are split off from the infinitives they direct, in which the infinitive is an event. This is because the ETCBC's strict structuralist methodology has defined the infinitive event as a clause, operating at a different hierarchical level than the phrase; phrases are defined as strictly non-predicative. In the framework of Construction Grammar (Goldberg, *Constructions*, 1995, Croft, *Radical Construction Grammar*, 2001) preferred by this study, these divisions are not necessary, and in fact hinder an accurate and comprehensive description. In Construction Grammar, no division is assumed between syntax and semantics, and thus the difference between a clause and a phrase is merely a difference in degree based on the two construction's forms and meanings, but it is not a fundamental difference in kind. Indeed, many phrases in this dataset refer to event-like nouns, which are from a syntactical perspective non-predicative; במות "in the death of" is a very frequent example, but other cases include ביום ישׁועה "in the day of salvation," which assumes a salvation event. If the difference between time phrases and time "clauses" is seen as merely an incremental difference rather than categorical, then one can apply a unified strategy in analyzing these cases. 

In [6]:
pd.set_option('display.max_colwidth', 0)  # configure DataFrame to show full notes with no truncation

In [7]:
head_anno = pd.read_csv('manual_curation/tp_heads_annotated.csv')
nhead_anno = pd.read_csv('manual_curation/tp_nheads_annotated.csv')

In [8]:
head_anno[head_anno['mark'] == 'b']

,token,(n)heads_token,freq,mark,note,ex_ref,ex,ex_node,ex_verse
214,ב.ה.שׁמים.ו.ב.ה.ארץ,ב.ב,1,b,bad phrase function (not a time phrase); good heads,Psalms 113:6,בַּשָּׁמַ֥יִם וּבָאָֽרֶץ׃,849296,הַֽמַּשְׁפִּילִ֥י לִרְאֹ֑ות בַּשָּׁמַ֥יִם וּבָאָֽרֶץ׃
251,ב.הר.ציון.ו.ב.ירושׁלם,ב.ב,1,b,not a time phrase; heads are good though,Joel 3:5,בְּהַר־צִיֹּ֨ון וּבִירוּשָׁלִַ֜ם,825329,וְהָיָ֗ה כֹּ֧ל אֲשֶׁר־יִקְרָ֛א בְּשֵׁ֥ם יְהוָ֖ה יִמָּלֵ֑ט כִּ֠י בְּהַר־צִיֹּ֨ון וּבִירוּשָׁלִַ֜ם תִּֽהְיֶ֣ה פְלֵיטָ֗ה כַּֽאֲשֶׁר֙ אָמַ֣ר יְהוָ֔ה וּבַ֨שְּׂרִידִ֔ים אֲשֶׁ֥ר יְהוָ֖ה קֹרֵֽא׃
710,כל.ימי.כורשׁ.מלך.פרס.ו.עד.מלכות.דריושׁ.מלך.פרס,ימי.מלכות,1,b,"This TP should contain an NP specified by a PP with waw connection. The heads algorithm appears to have selected past the prep in this case. That is the expected behavior, but it has a bad effect here. There may be other cases like this, which could be isolated by finding independent preps within NP phrases and checking their head assignments.",Ezra 4:5,כָּל־יְמֵ֗י כֹּ֚ורֶשׁ מֶ֣לֶךְ פָּרַ֔ס וְעַד־מַלְכ֖וּת דָּרְיָ֥וֶשׁ מֶֽלֶךְ־פָּרָֽס׃,881852,וְסֹכְרִ֧ים עֲלֵיהֶ֛ם יֹועֲצִ֖ים לְהָפֵ֣ר עֲצָתָ֑ם כָּל־יְמֵ֗י כֹּ֚ורֶשׁ מֶ֣לֶךְ פָּרַ֔ס וְעַד־מַלְכ֖וּת דָּרְיָ֥וֶשׁ מֶֽלֶךְ־פָּרָֽס׃
858,מ.ה.יום.ה.הוא.ו.הלאה,מ,1,b,This TP misses והלאה; my suspicion is that these words are being marked as spec or atr when they should be para.,1_Samuel 18:9,מֵהַיֹּ֥ום הַה֖וּא וָהָֽלְאָה׃ ס,738491,וַיְהִ֥י שָׁא֖וּל עֹויֵ֣ן אֶת־דָּוִ֑ד מֵהַיֹּ֥ום הַה֖וּא וָהָֽלְאָה׃ ס
859,מ.ה.יום.ה.הוא.ו.מעלה,מ,2,b,"This is like הלאה but with מעלה; m. should be picked up as a coordinated noun. To prove this, it would be helpful to show comparable examples with waw connections.",1_Samuel 30:25,מֵֽהַיֹּ֥ום הַה֖וּא וָמָ֑עְלָה,743564,וַיְהִ֕י מֵֽהַיֹּ֥ום הַה֖וּא וָמָ֑עְלָה וַיְשִׂמֶ֜הָ לְחֹ֤ק וּלְמִשְׁפָּט֙ לְיִשְׂרָאֵ֔ל עַ֖ד הַיֹּ֥ום הַזֶּֽה׃ פ
865,מ.יום.ה.שׁמיני.ו.הלאה,מ,1,b,another case of הלאה not encoded,Leviticus 22:27,מִיֹּ֤ום הַשְּׁמִינִי֙ וָהָ֔לְאָה,690126,שֹׁ֣ור אֹו־כֶ֤שֶׂב אֹו־עֵז֙ כִּ֣י יִוָּלֵ֔ד וְהָיָ֛ה שִׁבְעַ֥ת יָמִ֖ים תַּ֣חַת אִמֹּ֑ו וּמִיֹּ֤ום הַשְּׁמִינִי֙ וָהָ֔לְאָה יֵרָצֶ֕ה לְקָרְבַּ֥ן אִשֶּׁ֖ה לַיהוָֽה׃
949,מהלך.יום.אחד,מהלך,1,b,"“a walk of one day”—this does not seem to be temporal but rather distance expressed in terms of time. This function should be adjusted, probably to Adju, but possibly Loca.",Jonah 3:4,מַהֲלַ֖ךְ יֹ֣ום אֶחָ֑ד,828081,וַיָּ֤חֶל יֹונָה֙ לָבֹ֣וא בָעִ֔יר מַהֲלַ֖ךְ יֹ֣ום אֶחָ֑ד וַיִּקְרָא֙ וַיֹּאמַ֔ר עֹ֚וד אַרְבָּעִ֣ים יֹ֔ום וְנִֽינְוֵ֖ה נֶהְפָּֽכֶת׃
960,מן.ה.יום.ה.זה,מן,1,b,missing הלאה,Haggai 2:19,מִן־הַיֹּ֥ום הַזֶּ֖ה,831777,הַעֹ֤וד הַזֶּ֨רַע֙ בַּמְּגוּרָ֔ה וְעַד־הַגֶּ֨פֶן וְהַתְּאֵנָ֧ה וְהָרִמֹּ֛ון וְעֵ֥ץ הַזַּ֖יִת לֹ֣א נָשָׂ֑א מִן־הַיֹּ֥ום הַזֶּ֖ה אֲבָרֵֽךְ׃ ס
961,מן.ה.יום.ה.זה.ו.מעלה,מן,1,b,missing ומעלה,Haggai 2:15,מִן־הַיֹּ֥ום הַזֶּ֖ה וָמָ֑עְלָה,831733,וְעַתָּה֙ שִֽׂימוּ־נָ֣א לְבַבְכֶ֔ם מִן־הַיֹּ֥ום הַזֶּ֖ה וָמָ֑עְלָה מִטֶּ֧רֶם שֽׂוּם־אֶ֛בֶן אֶל־אֶ֖בֶן בְּהֵיכַ֥ל יְהוָֽה׃
962,מן.ה.יום.ה.זה.ו.מעלה.מ.יום.עשׂרים.ו.ארבעה.ל.ה.תשׁיעי.ל.מן.ה.יום,מן.ל,1,b,"missing מעלה. It’s also not clear why second ל is picked up as a head. It is not necessarily NOT a head, but this TP seems to have some complex syntax. Investigation needed.",Haggai 2:18,מִן־הַיֹּ֥ום הַזֶּ֖ה וָמָ֑עְלָה מִיֹּום֩ עֶשְׂרִ֨ים וְאַרְבָּעָ֜ה לַתְּשִׁיעִ֗י לְמִן־הַיֹּ֛ום,831763,שִׂימוּ־נָ֣א לְבַבְכֶ֔ם מִן־הַיֹּ֥ום הַזֶּ֖ה וָמָ֑עְלָה מִיֹּום֩ עֶשְׂרִ֨ים וְאַרְבָּעָ֜ה לַתְּשִׁיעִ֗י לְמִן־הַיֹּ֛ום אֲשֶׁר־יֻסַּ֥ד הֵֽיכַל־יְהוָ֖ה שִׂ֥ימוּ לְבַבְכֶֽם׃


In [9]:
nhead_anno[nhead_anno['mark']=='b']

,token,(n)heads_token,freq,mark,note,ex_ref,ex,ex_node,ex_verse
10,אחרי,NaN,4,b,no nhead,Nehemiah 3:30,אַחֲרָ֨יו,884413,אַחֲרָ֨יו הֶחֱזִ֜יק חֲנַנְיָ֣ה בֶן־שֶׁלֶמְיָ֗ה וְחָנ֧וּן בֶּן־צָלָ֛ף הַשִּׁשִּׁ֖י מִדָּ֣ה שֵׁנִ֑י ס אַחֲרָ֣יו הֶחֱזִ֗יק מְשֻׁלָּם֙ בֶּן־בֶּ֣רֶכְיָ֔ה נֶ֖גֶד נִשְׁכָּתֹֽו׃ ס
38,אחריה,NaN,1,b,no nhead,2_Chronicles 11:20,אַחֲרֶ֣יהָ,897487,וְאַחֲרֶ֣יהָ לָקַ֔ח אֶֽת־מַעֲכָ֖ה בַּת־אַבְשָׁלֹ֑ום וַתֵּ֣לֶד לֹ֗ו אֶת־אֲבִיָּה֙ וְאֶת־עַתַּ֔י וְאֶת־זִיזָ֖א וְאֶת־שְׁלֹמִֽית׃
39,אחריהם,NaN,2,b,no nhead,Genesis 48:6,אַחֲרֵיהֶ֖ם,668648,וּמֹולַדְתְּךָ֛ אֲשֶׁר־הֹולַ֥דְתָּ אַחֲרֵיהֶ֖ם לְךָ֣ יִהְי֑וּ עַ֣ל שֵׁ֧ם אֲחֵיהֶ֛ם יִקָּרְא֖וּ בְּנַחֲלָתָֽם׃
40,אחריהן,NaN,2,b,no nhead,Genesis 41:27,אַחֲרֵיהֶ֗ן,665677,וְשֶׁ֣בַע הַ֠פָּרֹות הָֽרַקֹּ֨ות וְהָרָעֹ֜ת הָעֹלֹ֣ת אַחֲרֵיהֶ֗ן שֶׁ֤בַע שָׁנִים֙ הֵ֔נָּה וְשֶׁ֤בַע הַֽשִׁבֳּלִים֙ הָרֵקֹ֔ות שְׁדֻפֹ֖ות הַקָּדִ֑ים יִהְי֕וּ שֶׁ֖בַע שְׁנֵ֥י רָעָֽב׃
41,אחריו,NaN,22,b,no nhead,Nehemiah 3:22,אַחֲרָ֛יו,884359,וְאַחֲרָ֛יו הֶחֱזִ֥יקוּ הַכֹּהֲנִ֖ים אַנְשֵׁ֥י הַכִּכָּֽר׃
42,אחריך,NaN,2,b,no nhead,1_Kings 3:12,אַחֲרֶ֖יךָ,754959,הִנֵּ֥ה עָשִׂ֖יתִי כִּדְבָרֶ֑יךָ הִנֵּ֣ה׀ נָתַ֣תִּי לְךָ֗ לֵ֚ב חָכָ֣ם וְנָבֹ֔ון אֲשֶׁ֤ר כָּמֹ֨וךָ֙ לֹא־הָיָ֣ה לְפָנֶ֔יךָ וְאַחֲרֶ֖יךָ לֹא־יָק֥וּם כָּמֹֽוךָ׃
59,ב.אחרית.מלכותם,אחרית,1,b,should be מלכותם,Daniel 8:23,בְאַחֲרִית֙ מַלְכוּתָ֔ם,879774,וּֽבְאַחֲרִית֙ מַלְכוּתָ֔ם כְּהָתֵ֖ם הַפֹּשְׁעִ֑ים יַעֲמֹ֛ד מֶ֥לֶךְ עַז־פָּנִ֖ים וּמֵבִ֥ין חִידֹֽות׃
113,ב.ה.חמשׁי,חמשׁי,1,b,this TP should be subsumed under the one immediately preceding it; the phrase boundary needs to be redrawn actually. Should pull comparable examples to validate this choice.,Ezekiel 20:1,בַּֽחֲמִשִׁי֙,812472,וַיְהִ֣י׀ בַּשָּׁנָ֣ה הַשְּׁבִיעִ֗ית בַּֽחֲמִשִׁי֙ בֶּעָשֹׂ֣ור לַחֹ֔דֶשׁ בָּ֧אוּ אֲנָשִׁ֛ים מִזִּקְנֵ֥י יִשְׂרָאֵ֖ל לִדְרֹ֣שׁ אֶת־יְהוָ֑ה וַיֵּשְׁב֖וּ לְפָנָֽי׃ ס
119,ב.ה.יום.ה.הוא.ב.יום,יום,1,b,"The second part of this TP, ביום, is separated from an infinitival clause that is part of it. The whole part is an appositional use. I should consider redrawing this TP, perhaps to include the inf. clause. Or I should consider creating a construction object that it can be subsumed under.",Ezekiel 38:18,בַּיֹּ֣ום הַה֗וּא בְּיֹ֨ום,819264,וְהָיָ֣ה׀ בַּיֹּ֣ום הַה֗וּא בְּיֹ֨ום בֹּ֥וא גֹוג֙ עַל־אַדְמַ֣ת יִשְׂרָאֵ֔ל נְאֻ֖ם אֲדֹנָ֣י יְהוִ֑ה תַּעֲלֶ֥ה חֲמָתִ֖י בְּאַפִּֽי׃
134,ב.ה.יום.ה.שׁמיני.ו.הלאה,יום,1,b,missing הלאה,Ezekiel 43:27,בַיֹּ֨ום הַשְּׁמִינִ֜י וָהָ֗לְאָה,820983,וִֽיכַלּ֖וּ אֶת־הַיָּמִ֑ים ס וְהָיָה֩ בַיֹּ֨ום הַשְּׁמִינִ֜י וָהָ֗לְאָה יַעֲשׂ֨וּ הַכֹּהֲנִ֤ים עַל־הַמִּזְבֵּ֨חַ֙ אֶת־עֹולֹֽותֵיכֶם֙ וְאֶת־שַׁלְמֵיכֶ֔ם וְרָצִ֣אתִי אֶתְכֶ֔ם נְאֻ֖ם אֲדֹנָ֥י יְהֹוִֽה׃ ס


In [105]:
nhead_anno[nhead_anno['(n)heads_token'] == 'ראשׁ']

,token,(n)heads_token,freq,mark,note,ex_ref,ex,ex_node,ex_verse
686,ל.ראשׁ.אשׁמרות,ראשׁ,1,b,head should be אשׁמרות? But that also depends on whether “head” should be treated prepositionally…I have gone back and forth on this.,Lamentations 2:19,לְרֹאשׁ֙ אַשְׁמֻרֹ֔ות,873192,ק֣וּמִי׀ רֹ֣נִּי בַלַּ֗יְלָה לְרֹאשׁ֙ אַשְׁמֻרֹ֔ות שִׁפְכִ֤י כַמַּ֨יִם֙ לִבֵּ֔ךְ נֹ֖כַח פְּנֵ֣י אֲדֹנָ֑י שְׂאִ֧י אֵלָ֣יו כַּפַּ֗יִךְ עַל־נֶ֨פֶשׁ֙ עֹֽולָלַ֔יִךְ הָעֲטוּפִ֥ים בְּרָעָ֖ב בְּרֹ֥אשׁ כָּל־חוּצֹֽות׃ ס
789,מ.ראשׁ,ראשׁ,4,g,NaN,Isaiah 40:21,מֵרֹ֖אשׁ,782700,הֲלֹ֤וא תֵֽדְעוּ֙ הֲלֹ֣וא תִשְׁמָ֔עוּ הֲלֹ֛וא הֻגַּ֥ד מֵרֹ֖אשׁ לָכֶ֑ם הֲלֹוא֙ הֲבִ֣ינֹתֶ֔ם מֹוסְדֹ֖ות הָאָֽרֶץ׃
790,מ.ראשׁ.מ.קדמי.ארץ,ראשׁ,1,g,NaN,Proverbs 8:23,מֵרֹ֗אשׁ מִקַּדְמֵי־אָֽרֶץ׃,862813,מֵ֭עֹולָם נִסַּ֥כְתִּי מֵרֹ֗אשׁ מִקַּדְמֵי־אָֽרֶץ׃


## Exploring Examples

In [2]:
T.text(E.nhead.t(873192))

'אַשְׁמֻרֹ֔ות '

In [76]:
F.prs.v(35963)

'absent'

### Adjacent TP's

Adjacent time phrases may indicate cases where a TP has been cleft into two, whereas it is more frequently kept together. A simple query is made for those cases here...

In [7]:
cleft_tp = A.search('''

clause
    phrase function=Time
    <: phrase function=Time

''')

#A.show(cleft_tp, condenseType='clause')

  0.53s 72 results


## Orphaned Time Phrases

This search checks all time phrase surface forms against phases marked with "Adju" (adjunct) to identify potential candidates for orphaned time phrases.